In [3]:
import json
import subprocess

def get_json_segments(file_path: str, interval: int) -> list:

    # Define the command to run with jq
    command = [
        'jq', 
        f'.[:{interval}]', 
        file_path
    ]

    # Run the command and capture its output
    output = subprocess.check_output(command)

    # Decode the output from bytes to a UTF-8 string
    output_str = output.decode('utf-8')

    # Parse the output as a JSON object
    result = json.loads(output_str)

    return result

In [3]:
from collections import defaultdict

# file_path = "twitter-data-small.json"
file_path = "smallTwitter_2.json"

j = json.load(open(file_path, "r"))
print(type(j))

d = defaultdict(lambda:0)
for obj in j:
    author_id = obj["data"]["author_id"]
    d[author_id] += 1

s = sorted(d.items(), key = lambda x: x[1], reverse=True)

sum = 0
for i in s:
    if s.index(i) < 10:
        print(i)
    sum += i[1]

print("Total number of tweets: ", sum)

<class 'list'>
('58194459', 605)
('778785859030003712', 492)
('820431428835885059', 462)
('1381214332114010114', 429)
('765638249125703681', 394)
('1135850507606478848', 393)
('1624641818', 378)
('67308556', 362)
('1250331934242123776', 317)
('7050962', 310)
Total number of tweets:  119160


715 tweets ~ 1.5 MB
119160 tweets ~ 242 MB
- The increase in linear.

In [5]:
print(j[0].keys())

dict_keys(['_id', '_rev', 'data', 'includes', 'matching_rules'])


# Explore format in .data.places.full_name

In [6]:
for obj in j:
    target = obj["includes"]["places"]
    # print(target[0]["full_name"])
    if len(target) > 1:
        print(f"\nThis is from the same tweet above. {target[1]}")

In [7]:
for obj in j:
    target = obj["includes"]["places"]
    if len(target[0]["full_name"].split(", ")) > 2:
        print(target[0]["full_name"])

for obj in j:
    target = obj["includes"]["places"]
    if len(target[0]["full_name"].split(" - ")) > 2:
        print(target[0]["full_name"])

for obj in j:
    target = obj["includes"]["places"]
    if len(target) == 0:
        print("Found empty `places`")

In [8]:
for obj in j:
    target = obj["includes"]["places"]
    if len(target[0]["full_name"].split(", ")) == 1:
        print(target[0]["full_name"])

Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Ourimbah
Sydney Olympic Park
Woolwich
Meroo Meadow
Meroo Meadow
Meroo Meadow
Meroo Meadow
Blairgowrie
North Melbourne
Ellis Beach
Aldinga Beach
Henley Beach South
City Beach
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Bondi Beach
Parramatta
Sydney Olympic Park
Winmalee
Main Beach
Port Noarlunga
Coles
Zuccoli
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Australia
Aust

From the output above, we know
- No tweets here has more than one obj in `.data.places`
- Locations are separated by at most 1 comma
- There are at most 2 places in a location, separated by a ` - `
- All tweets have at least one obj in `places`.
- Most of location info in `full_name` composed of `suburb, state/suburb`

Location counting process:
#TODO: Documentation after testing.

*Remebmer to lower case!

Output json:

{
    "author_id": {
        "total_count": "",
        "capital_counts": {
            "1gsyd": 0,
            "2gmel": 0,
            "3gbri": 0,
            "4gade": 0,
            "5gper": 0,
            "6ghob": 0
        }
    }
}

- As Darwin not in spec, ignored in task one.

For coding convenience:
{
    "author_id": np.zeros(7)
}


# Explore twitter json and sal json with regex

In [12]:
import re

sal = json.load(open('sal.json', 'r'))
regex_sal = re.compile(r"""
    ([^()]+) {1}  # Get the pattern before a parenthesis shows up
    (
        \( 
            # (\w+-\w+ | \w+\s\w+ | \w+) 
            # (\s-\s)? 
            # ([^-()]+)? 
            ([^-\s()]+ ((-|\s)[^-\s()]+)+ | [^-\s()]+) 
            (\s-\s)? 
            ([^-()]+)? 
        \) 
    ) ?
    # Aims to limit to at most one set of parenthesis.
""", re.X)

suburb_alias_set = set()
count = 0
for k in sal.keys():
    match = regex_sal.search(k)
    # if match.groups()[2] and match.groups()[6]:
    if True:
        count += 1
        # print("The whole key is: ", k)
        print("{}, {}, {}".format(match.groups()[0], match.groups()[2], match.groups()[6]))
        suburb_alias_set.add((match.groups()[0], match.groups()[2]))
        for group in match.groups():
            print(group)
        # print("\n")
    # if match is None:
    #     print(k)

if count == len(suburb_alias_set):
    print("Combination of suburb and alias is unique")

abbotsbury, None, None
abbotsbury
None
None
None
None
None
None
abbotsford , nsw, None
abbotsford 
(nsw)
nsw
None
None
None
None
acacia gardens, None, None
acacia gardens
None
None
None
None
None
None
agnes banks, None, None
agnes banks
None
None
None
None
None
None
airds, None, None
airds
None
None
None
None
None
None
alexandria, None, None
alexandria
None
None
None
None
None
None
alfords point, None, None
alfords point
None
None
None
None
None
None
alison , central coast, nsw
alison 
(central coast - nsw)
central coast
 coast
 
 - 
nsw
allambie heights, None, None
allambie heights
None
None
None
None
None
None
allawah, None, None
allawah
None
None
None
None
None
None
ambarvale, None, None
ambarvale
None
None
None
None
None
None
angus, None, None
angus
None
None
None
None
None
None
annandale , nsw, None
annandale 
(nsw)
nsw
None
None
None
None
annangrove, None, None
annangrove
None
None
None
None
None
None
appin , nsw, None
appin 
(nsw)
nsw
None
None
None
None
arcadia , nsw, None
arca

The above result shows that the regex is able to capture the desired pattern.
The result below shows a rare case where " - " appears before brackets.

In [13]:
for k in sal.keys():
    match = regex_sal.search(k)
    info_2 = match.groups()[2]
    info_3 = match.groups()[6]

    if "(" in k:
        assert(info_2 is not None)
    if " - " in k:
        if info_3 is None:
            print("k: ", k)
            for g in match.groups():
                print(g)
        
    if "(" not in k:
        assert(info_2 is None) 

k:  murputja - nyapari
murputja - nyapari
None
None
None
None
None
None


In [2]:
import ijson

# with open("smallTwitter.json", "rb") as f:
with open("twitter-data-small.json", "rb") as f:
    for record in ijson.items(f, "item"):
        print(record["includes"]["places"][0]["full_name"])

Australia
Australia
New South Wales, Australia
New South Wales, Australia
New South Wales, Australia
New South Wales, Australia
New South Wales, Australia
New South Wales, Australia
Central Coast, New South Wales
Central Coast, New South Wales
Central Coast, New South Wales
Central Coast, New South Wales
New South Wales, Australia
New South Wales, Australia
New South Wales, Australia
New South Wales, Australia
New South Wales, Australia
Central Coast, New South Wales
New South Wales, Australia
Central Coast, New South Wales
New South Wales, Australia
New South Wales, Australia
Nelson Bay - Corlette, New South Wales
New South Wales, Australia
Eveleigh, Sydney
Kirribilli, Sydney
Macquarie Park, Sydney
Picton, New South Wales
Picton, New South Wales
Picton, New South Wales
Sydney, New South Wales
Sydney, New South Wales
Sydney, New South Wales
Sydney, New South Wales
Sydney, New South Wales
Sydney, New South Wales
Sydney, New South Wales
Sydney, New South Wales
Sydney, New South Wales
Syd

## Prototype Task 1 output

In [5]:
import pandas as pd
import numpy as np

In [9]:
arr_template = np.ones(9, dtype=np.int8)

data = {
    "123": arr_template,
    "234": arr_template,
    "356": arr_template,
    "465": arr_template,
    "5343": arr_template
}
df = pd.DataFrame(data).T.iloc[:, :1].sort_values(by=[0], ascending=False)
# df = pd.DataFrame(data).T
# df.iloc[2, 0] = 3
# df.sort_values(by=[0], ascending=False, inplace=True)
df

,0
123,1
234,1
356,1
465,1
5343,1


In [10]:
df.rename(columns={0: "Number of Tweets Made"}, inplace=True)
df.index.name='Author ID'
df.reset_index(inplace=True)
df.set_index([pd.Index([f"#{i+1}" for i in range(5)])], inplace=True)
df

,Author ID,Number of Tweets Made
#1,123,1
#2,234,1
#3,356,1
#4,465,1
#5,5343,1


In [11]:
df.index.name = 'Rank'
df

,Author ID,Number of Tweets Made
Rank,,
#1,123,1
#2,234,1
#3,356,1
#4,465,1
#5,5343,1


In [12]:
def explore_df_end(df, compare_keys, default_end=9):
    """
        compare_keys: 
            A list of strings that specify the fields for ranking the df row.
    """
    is_equal_priority = True
    i = default_end
    while is_equal_priority:
        for key in compare_keys:
            if df[key][i] != df[key][i+1]:
                break
        i += 1
        if i == len(df) - 1:
            break
    return df.iloc[:i+1, :]

explore_df_end(df, ["Number of Tweets Made"], default_end=0)

,Author ID,Number of Tweets Made
Rank,,
#1,123,1
#2,234,1
#3,356,1
#4,465,1
#5,5343,1


In [3]:
big_arr = np.ones((10**7,2))
df_big = pd.DataFrame(big_arr)
df_big

,0,1
0,1.0,1.0
1,1.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,1.0
...,...,...
9999995,1.0,1.0
9999996,1.0,1.0
9999997,1.0,1.0
9999998,1.0,1.0


In [4]:
df_big.sort_values(by=[0])

,0,1
0,1.0,1.0
6666662,1.0,1.0
6666663,1.0,1.0
6666664,1.0,1.0
6666665,1.0,1.0
...,...,...
3333335,1.0,1.0
3333336,1.0,1.0
3333337,1.0,1.0
3333330,1.0,1.0


In [17]:
df_big[1]

0          1.0
1          1.0
2          1.0
3          1.0
4          1.0
          ... 
9999995    1.0
9999996    1.0
9999997    1.0
9999998    1.0
9999999    1.0
Name: 1, Length: 10000000, dtype: float64